In [15]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

### Labor Productivity Process
In the Aiyagari (1994) model, household (log-) labor productivity, $\varepsilon_t$, follows an AR(1) process with persistence parameter $\rho$. To simplify the model solution, we approximate the AR(1) process using a discrete Markov chain. This approximation is performed using "Tauchen's method," a standard approach for discretizing continuous state spaces. The function `approx_markov` is used to generate the corresponding Markov chain.

In [16]:
def approx_markov(rho, sigma_u, m=3, n=7):
    """
    Computes the Markov matrix associated with a discretized version of
    the linear Gaussian AR(1) process

        y_{t+1} = rho * y_t + u_{t+1}

    according to Tauchen's method.  Here {u_t} is an iid Gaussian
    process with zero mean.

    Parameters
    ----------
    rho : scalar(float)
        The autocorrelation coefficient
    sigma_u : scalar(float)
        The standard deviation of the random process
    m : scalar(int), optional(default=3)
        The number of standard deviations to approximate out to
    n : scalar(int), optional(default=7)
        The number of states to use in the approximation

    Returns
    -------

    x : array_like(float, ndim=1)
        The state space of the discretized process
    P : array_like(float, ndim=2)
        The Markov transition matrix where P[i, j] is the probability
        of transitioning from x[i] to x[j]

    """
    F = scipy.stats.norm(loc=0, scale=sigma_u).cdf

    # standard deviation of y_t
    std_y = np.sqrt(sigma_u**2 / (1-rho**2))

    # top of discrete state space
    x_max = m * std_y

    # bottom of discrete state space
    x_min = - x_max

    # discretized state space
    x = np.linspace(x_min, x_max, n)

    step = (x_max - x_min) / (n - 1)
    half_step = 0.5 * step
    P = np.empty((n, n))

    for i in range(n):
        P[i, 0] = F(x[0]-rho * x[i] + half_step)
        P[i, n-1] = 1 - F(x[n-1] - rho * x[i] - half_step)
        for j in range(1, n-1):
            z = x[j] - rho * x[i]
            P[i, j] = F(z + half_step) - F(z - half_step)

    return x, P

In the next step we generate the markov chain approximation of an AR(1) process with `neta=7` realizations, persistence $\rho=0.965$, and standard deviation $\sigma=0.25$. The realizations of the shock are given by `y` and the transition matrix is given by `yprob`.

In [17]:
neta = 7 # number of realizations
rho_e = 0.965 # persistence
sig_e = 0.25 # standard deviation

# discretize the AR-process into a markov chain
y, yprob = approx_markov(rho_e, sig_e, 3, neta)

# normalization
y = np.exp(y)

Next we compute the stationary distribution of the process, and the mean labor supply under the AR(1) process.

In [18]:
# find the stationary distribution of the markov chain
S, U = scipy.linalg.eig(yprob.T)
pistar = np.array(U[:, np.where(np.abs(S - 1.) < 1e-8)[0][0]].flat)
pistar = pistar / np.sum(pistar)

# aggregate labor supply
var_x = sig_e**2.0 / (1.0 - rho_e**2.0)
labor = np.exp(var_x / 2.0)

### Exercise